In [ ]:
%config Completer.use_jedi = False
import matplotlib
matplotlib.use('TkAgg')


In [ ]:
import os
import numpy as np
import pandas as pd
from operator import truediv, mul
import array
from matplotlib import rcParams
import matplotlib.pyplot as plt
from matplotlib import lines


In [ ]:
import seaborn as sns

In [ ]:
def resampling(map_data, long=False):
    v = []
    area = []
    ent = []
    cnt = 0
    for i in map_data.item()['time']:
        v.append(float(i.to_sec()))
        if(map_data.item()['area'][cnt] != 0):
            area.append(map_data.item()['area'][cnt])
        else:
            area.append(0.0025)
        ent.append(map_data.item()['total_entropy'][cnt])
        cnt += 1
    d = pd.DataFrame(columns=['ts','area','ent'])

    d.loc[:,'ts'] = v
    d.loc[:,'area'] = area
    d.loc[:,'ent'] = ent
    d = d.set_index('ts')
    d.index = pd.to_datetime(d.index, unit='s')
    tmp = d.resample('2S')[['area']].agg(lambda x : np.nan if x.count() == 0 else x.idxmax()).ffill()
    df = d.loc[tmp['area']]
    cnt = 0
    if (not long):
        while (len(df) < 300):
            new_row = pd.DataFrame(columns=['area','ent'])
            new_row.loc[0 + cnt / 1000.0,'area'] = 0.0025
            new_row.loc[0 + cnt / 1000.0,'ent'] = 1
            cnt += 1
            df = pd.concat([new_row, df])
    else:
        while (len(df) < 375):
            new_row = pd.DataFrame(columns=['area','ent'])
            new_row.loc[0 + cnt / 1000.0,'area'] = 0.0025
            new_row.loc[0 + cnt / 1000.0,'ent'] = 1
            cnt += 1
            df = pd.concat([new_row, df])
        
    return df

In [ ]:
def get_rmse(occupancy, gt):
    oc = []
    se = 0
    for l in occupancy:
        p = float(l)
        oc.append(p)
    oc_gt = []
    for l in gt:
        p = float(l)
        oc_gt.append(p)
    for l in range(len(oc)):
        if(oc[l] == -1):
            oc[l] = 50
        if(oc_gt[l] == -1):
            oc_gt[l] = 50
        se += (oc[l] - oc_gt[l])**2
    mse = se/len(oc)
    return mse**0.5

def get_rmse_known(occupancy, gt):
    oc = []
    se = 0
    for l in occupancy:
        p = float(l)
        oc.append(p)
    oc_gt = []
    for l in gt:
        p = float(l)
        oc_gt.append(p)
    cnt = 0.0
    for l in range(len(oc)):
        if (oc[l] == -1):
            if (oc_gt[l] != -1):
                oc[l] = 50
                cnt += 1
                se += (oc[l] - oc_gt[l])**2
        else:
            if(oc_gt == -1):
                oc_gt[l] = 100 - oc[l]
            cnt += 1
            se += (oc[l] - oc_gt[l])**2
    mse = se/cnt
    return mse**0.5

def bac_manual(occupancy, gt):
    oc = []
    se = 0
    for l in occupancy:
        p = float(l)
        oc.append(p)
    oc_gt = []
    for l in gt:
        p = float(l)
        oc_gt.append(p)
    tot_free = 0
    tot_occ = 0
    corr_free = 0
    corr_occ = 0
    tot_unk = 0
    corr_unk = 0
    for l in range(len(oc)):
        if oc_gt[l] == 0:
            tot_free += 1
            if oc[l] == oc_gt[l]:
                corr_free += 1
        elif oc_gt[l] == 100:
            tot_occ += 1
            if oc[l] == oc_gt[l]:
                corr_occ += 1
        elif oc_gt[l] == -1:
            tot_unk += 1
            if oc[l] == oc_gt[l]:
                corr_unk += 1
    return [1/3.0*(corr_free/tot_free + corr_occ/tot_occ + corr_unk/tot_unk),corr_free/tot_free,corr_occ/tot_occ, corr_unk/tot_unk]


In [ ]:
def get_data(folder, columns, kind, df, low = 0, r = 10, ate = False, saving_results = False):
    color = ['tab:blue','tab:green','tab:red','tab:orange','tab:purple','tab:brown','tab:pink','tab:gray','tab:olive','tab:cyan','k']
    final_folders = []
    if 'E1' in folder:
        for i in columns:
            for k in kind:
                final_folders.append(os.path.join(folder, i+"_E1_"+k))
    else:
        for i in columns:
            for k in kind:
                final_folders.append(os.path.join(folder, i+'_E2_'+k))

    cnt = 0
    
#     fig1, (ax1, ax2) = plt.subplots(1,2)    
    fig_map, (ax_map) = plt.subplots(1)
    fig_exp, (ax_exp) = plt.subplots(1)
#     fig_ent, (ax_ent) = plt.subplots(1)
#     fig_area, (ax_area) = plt.subplots(1)

    
    for i in final_folders:
        tot_score = []
        tot_BAC =  []
        tot_BAC_free = []
        tot_BAC_occ = []
        if ate:
            tot_TRPE = []
            tot_RRPE = []
            tot_ATE = []
        cov_vec = []
        tot_lc = []
        glob_lc = []
        loc_lc = []
        tot_area = []
        area_bucketed = []
        entropy_bucketed = []
        pose_cov_vec = []
        final_map_rmse = []
        bac_manually = []
        bac_manually_unk = []
        bac_manually_free = []
        bac_manually_occ = []
        path_len =[]
        filtered_path_df = []
        wheel_tot = []
        for j in range(low,r):
            current = os.path.join(i,str(j))
#             print(current)
            if (os.path.isfile(os.path.join(current,'map_data.npy'))):
                map_data = np.load(os.path.join(current,'map_data.npy'), allow_pickle=True, encoding="latin1")
                system_data = np.load(os.path.join(current,'feat.npy'), allow_pickle=True, encoding="latin1")
                wheel_data = np.load(os.path.join(current,'wheels.npy'), allow_pickle=True, encoding="latin1")
                general_results = open(os.path.join(current,'general_results.txt'), 'r')
                gt = open(os.path.join(folder,'gt_occupancy.txt'), 'r')
                occupancy = open(os.path.join(current,'occupancy.txt'), 'r')
                occupancy_prob = open(os.path.join(current,'occupancy.txt'), 'r')
                poses = open(os.path.join(current,'poses.g2o'), 'r')
                tot_lc.append(system_data.item()['total'][-1])
                glob_lc.append(system_data.item()['global'][-1])
                loc_lc.append(system_data.item()['local'][-1])
                tot_area.append(map_data.item()['area'][-1])
                wheel_tot.append(wheel_data.item()['tot_wheel'][-1])


                final_map_rmse.append(get_rmse(occupancy,gt))
                gt = open(os.path.join(folder,'gt_occupancy.txt'), 'r')
                occupancy = open(os.path.join(current,'occupancy.txt'), 'r')
                
                gt = open(os.path.join(folder,'gt_occupancy.txt'), 'r')
                occupancy = open(os.path.join(current,'occupancy.txt'), 'r')
                bac_m = bac_manual(occupancy,gt)
                bac_manually.append(bac_m[0])
                bac_manually_unk.append(bac_m[-1])
                bac_manually_free.append(bac_m[1])
                bac_manually_occ.append(bac_m[2])
                
                # binning the entropies and the area explored wrt 2-seconds time frames
                resampled = resampling(map_data, "_L" in current)
                if ("_L" in current):
                    area_bucketed.append(resampled[-375:].loc[:,'area'])            
                    entropy_sliced = resampled[-375:].loc[:,'ent']       
                else:
                    area_bucketed.append(resampled[-300:].loc[:,'area'])            
                    entropy_sliced = resampled[-300:].loc[:,'ent']       
                a = [0.0025 for i in range(len(area_bucketed[-1]))]
                res = list(map(truediv, area_bucketed[-1], a))
                res2 = list(map(truediv, entropy_sliced, res))
                print(res2[-1])
                entropy_bucketed.append(res2)

                # ATE - slam metric
                if ate:
                    path_gt = open(os.path.join(current,'rtabmap_odom.txt'),'r')
                    prev = [0,0,0]
                    first = True
                    path_df = pd.DataFrame(columns=['time','len'])
                    
                    for l in path_gt:
                        p = l.split()
                        if first:
                            prev = [float(p[1]), float(p[2])]
                            path_df.loc[p[0],'time'] = float(p[0])
                            path_df.loc[p[0],'len'] = 0
                            cum_dist = 0
                            first = False
                        else:
                            cum_dist += ((prev[0]-float(p[1]))**2+(prev[1]-float(p[2]))**2)**0.5
                            
                            path_df.loc[p[0],'time'] = float(p[0])
                            path_df.loc[p[0],'len'] = cum_dist
                            
                            prev = [float(p[1]), float(p[2])]
                            
                    path_len.append(cum_dist)
                    path_df.index = pd.to_datetime(path_df.index, unit='s')
                    tmp_filtered_path_df = path_df.resample('2S').max().ffill()
                    if ("_L" not in current):
                        norm = 0
                        while (len(tmp_filtered_path_df) < 300):

                            new_row = pd.DataFrame(columns=['time','len'])
                            new_row.loc[0,'time'] = 0+ norm/1000.0
                            new_row.loc[0,'len'] = 0.00001
                            new_row = new_row.set_index('time')
                            new_row.index = pd.to_datetime(new_row.index, unit='s')
                            norm += 1
                            tmp_filtered_path_df = pd.concat([new_row, tmp_filtered_path_df])
                        filtered_path_df.append(tmp_filtered_path_df[-300:]['len'])
                    else:
                        norm = 0
                        while (len(tmp_filtered_path_df) < 375):

                            new_row = pd.DataFrame(columns=['time','len'])
                            new_row.loc[0,'time'] = 0 + norm/1000.0
                            norm += 1
                            new_row.loc[0,'len'] = 0.00001
                            new_row = new_row.set_index('time')
                            new_row.index = pd.to_datetime(new_row.index, unit='s')
                            tmp_filtered_path_df = pd.concat([new_row, tmp_filtered_path_df])
                            
                        filtered_path_df.append(tmp_filtered_path_df[-375:]['len'])

                    ate = open(os.path.join(current,'ate.txt'), 'r')
                    for l in ate:
                        if 'rmse' in l:
                            tot_ATE.append(float(l.split()[-2]))
                    
                    rpe = open(os.path.join(current,'rpe.txt'), 'r')
                    for l in rpe:
                        if 'translational' in l and 'rmse' in l:
                            tot_TRPE.append(float(l.split()[-2]))
                        elif 'rotational' in l and 'rmse' in l:
                            tot_RRPE.append(float(l.split()[-2]))
                            
                    odom = np.load(os.path.join(current,'odom.npy'), allow_pickle=True, encoding="latin1")
                    for k in range(len(odom.item()['time'])):
                        if (odom.item()['time'][-1].to_sec() - odom.item()['time'][k].to_sec() <= 603):
                            m = np.array(odom.item()['cov'][k]).reshape((6,6))
                            eigen = np.linalg.eig(m)
                            pose_cov_vec.append(np.exp(1/6.0 * np.sum(np.log(eigen[0]))).real)
                # collect map scores from general_results file
                for l in general_results:
                    if 'map_score' in l:
                        tot_score.append(float(l.split()[-1]))
                    if 'BAC score' in l:
                        tot_BAC.append(float(l.split()[-1]))
                    if 'correct_free_ratio' in l:
                        tot_BAC_free.append(float(l.split()[-1]))
                    if 'correct_occ_ratio' in l:
                        tot_BAC_occ.append(float(l.split()[-1]))

                # collect link variances of the graph
                for l in poses:
                    if 'EDGE' in l:
                        s = l.split()
                        m = np.array([[float(s[-6]), float(s[-5]), float(s[-4])],
                                      [float(s[-5]), float(s[-3]), float(s[-2])],
                                      [float(s[-4]), float(s[-2]), float(s[-1])]]
                                    )
                        cov = np.linalg.inv(m)
                        eigen = np.linalg.eig(cov)
                        cov_vec.append(np.exp(1/3.0 * np.sum(np.log(eigen[0]))).real)
                        #cov_vec.append(np.linalg.det(cov))
        if(len(tot_score)>0):
#             df.loc['map score avg',i.split('/')[-1]] = 'mean: ' + str(np.mean(tot_score)) + ' ' + ' std: ' + str(np.std(tot_score))
            if (saving_results):
                df.loc['BAC',i.split('/')[-1]] = 'mean: ${:.2f}$ std: ${:.2f}$'.format(round(np.mean(bac_manually),2),round(np.std(bac_manually),2))
            else:
                df.loc['BAC',i.split('/')[-1]] = 'mean: {:.2f} std: {:.2f}'.format(round(np.mean(bac_manually),2),round(np.std(bac_manually),2))
                
#             df.loc['BAC-free',i.split('/')[-1]] = 'mean: ' + str(np.mean(bac_manually_free)) + ' ' + ' std: ' + str(np.std(bac_manually_free))
#             df.loc['BAC-occupied',i.split('/')[-1]] = 'mean: ' + str(np.mean(bac_manually_occ)) + ' ' + ' std: ' + str(np.std(bac_manually_occ))
#             df.loc['BAC-unk',i.split('/')[-1]] = 'mean: ' + str(np.mean(bac_manually_unk)) + ' ' + ' std: ' + str(np.std(bac_manually_unk))
#             df.loc['tot_lc',i.split('/')[-1]] =  'mean: ' + str(np.mean(tot_lc)) + ' ' + ' std: ' + str(np.std(tot_lc))
#             df.loc['tot_area',i.split('/')[-1]] =  'mean: ' + str(np.mean(tot_area)) + ' ' + ' std: ' + str(np.std(tot_area))
#             df.loc['tot_wheel',i.split('/')[-1]] =  'mean: ' + str(np.mean(wheel_tot)) + ' ' + ' std: ' + str(np.std(wheel_tot))
#             df.loc['final_entropy_norm',i.split('/')[-1]] =  'mean: {:.2f} std: {:.2f}'.format(round(np.mean(entropy_bucketed, axis = 0)[-1],2),round(np.std(entropy_bucketed, axis = 0)[-1],2))
#             df.loc['final_rmse',i.split('/')[-1]] =  'mean: ' + str(np.mean(final_map_rmse)) + ' ' + ' std: ' + str(np.std(final_map_rmse))
            if ate:
                if (saving_results):
                    df.loc['ATE',i.split('/')[-1]] =  'mean: ${:.3f}$ std: ${:.3f}$'.format(round(np.mean(tot_ATE),3),round(np.std(tot_ATE),3))
                else:
                    df.loc['ATE',i.split('/')[-1]] =  'mean: {:.3f} std: {:.3f}'.format(round(np.mean(tot_ATE),3),round(np.std(tot_ATE),3))
                    
#                 df.loc['tot_TRPe',i.split('/')[-1]] =  'mean: ' + str(np.mean(tot_TRPE)) + ' ' + ' std: ' + str(np.std(tot_TRPE))
#                 df.loc['tot_RRPe',i.split('/')[-1]] =  'mean: ' + str(np.mean(tot_RRPE)) + ' ' + ' std: ' + str(np.std(tot_RRPE))
#                 df.loc['path_len',i.split('/')[-1]] =  'mean: ' + str(np.mean(path_len)) + ' ' + ' std: ' + str(np.std(path_len))
#                 df.loc['area over path',i.split('/')[-1]] =  'mean: ' + str(np.mean(tot_area)/np.mean(path_len))
                lc_over_path = list(map(truediv, tot_lc, path_len))
                if (saving_results):
                    df.loc['Loops per m',i.split('/')[-1]] =  'mean: ${:.2f}$ std: ${:.2f}$'.format(round(np.mean(lc_over_path),2),round(np.std(lc_over_path),2))
                else:
                    df.loc['Loops per m',i.split('/')[-1]] =  'mean: {:.2f} std: {:.2f}'.format(round(np.mean(lc_over_path),2),round(np.std(lc_over_path),2))
                
            panda = pd.DataFrame(columns=['time','norm_entropy','area','path_len','exp'])
            cnt_exp = 0
            for tries in range(len(entropy_bucketed)):
                for result in range(len(entropy_bucketed[tries])):
                    panda.loc[cnt_exp,'norm_entropy'] = entropy_bucketed[tries][result]
                    panda.loc[cnt_exp,'time'] = result*2
                    if ('E2' in current):
                        panda.loc[cnt_exp,'area'] = area_bucketed[tries][result]/204.0 #area_bucketed[tries][result]
                    else:
                        panda.loc[cnt_exp,'area'] = area_bucketed[tries][result]/183.0 #area_bucketed[tries][result]
                    panda.loc[cnt_exp,'path_len'] = filtered_path_df[tries][result]
                    cnt_exp += 1
            panda.norm_entropy = panda.norm_entropy.astype(float)
            panda.path_len = panda.path_len.astype(float)
            panda.area = panda.area.astype(float)
#             sns.lineplot(data=panda, x='time', ci=None, y="norm_entropy", ax=ax_ent, color=color[cnt], label=i.split('/')[-1])
            
            ax_map_split = ax_map.twinx()
            ax_map_split.set_ylabel('trajectory length [m]')
            ax_map_split.set_ylim([0,120])
            sns.lineplot(data=panda, x='time', ci=None, y="area", ax=ax_map, color=color[cnt], label=i.split('/')[-1].replace('E1_','').replace('E2_','').replace('_S',''))
            sns.lineplot(data=panda, x='time', ci=None, y="path_len", ax=ax_map_split, color=color[cnt])

#             ax_ent.plot(range(len(entropy_bucketed[0])), np.mean(entropy_bucketed, axis = 0),color=color[cnt], label=i.split('/')[-1])
#             ax_map.plot(range(len(entropy_bucketed[0])), np.mean(area_bucketed, axis = 0),color=color[cnt], label=i.split('/')[-1])
            if ('E2' in current):
                ax_exp.plot(np.mean(filtered_path_df,axis=0), np.mean(area_bucketed, axis = 0)/204.0,color=color[cnt], label=i.split('/')[-1].replace('E2_','').replace('_S',''))
            else:
                ax_exp.plot(np.mean(filtered_path_df,axis=0), np.mean(area_bucketed, axis = 0)/183.0,color=color[cnt], label=i.split('/')[-1].replace('E1_','').replace('_S',''))
#             ax_area.plot(range(len(entropy_bucketed[0]))[10:], list(map(truediv,np.mean(filtered_path_df, axis = 0), np.mean(area_bucketed, axis = 0)))[10:],color=color[cnt], label=i.split('/')[-1])
            ax_exp.plot(np.mean(filtered_path_df,axis=0), np.mean(entropy_bucketed, axis=0),color=color[cnt])
            ax_exp.set_ylim([0,1])
            ax_exp.set_ylabel('explored %')
            ax_exp.set_xlabel('trajectory length [m]')
            sec_y = ax_exp.secondary_yaxis('right')
            sec_y.set_ylabel('normalized entropy')
            sec_y.set_ylim([0,1])
            ax_exp.legend()
            
#             ax_ent.legend()

            ax_map.set_ylabel('explored %')
            ax_map.set_xlabel('time [s]')
            ax_map.set_ylim([0,1])
            ax_map.legend()

#             ax_area.legend()

            # plot CDF
#             edges = compute_bin_size(cov_vec)
#             n, bins, patches = ax1.hist(cov_vec, edges, cumulative=True, density=True, histtype='step', label=i.split('/')[-1], color=color[cnt])
#             patches[0].set_xy(patches[0].get_xy()[:-1])
#             #ax1.plot(bins, n,  label=columns[cnt],c=np.random.rand(3,) )
#             if ate:
#                 edges = compute_bin_size(pose_cov_vec)
#                 n, bins, patches = ax2.hist(pose_cov_vec, edges, cumulative=True, density=True, histtype='step', label=i.split('/')[-1], color=color[cnt])
#                 patches[0].set_xy(patches[0].get_xy()[:-1])
                #ax2.plot(bins[1:], n,  ,c=np.random.rand(3,) )
            cnt += 1
#             print(tot_area)
            print(len(tot_lc))
            print('---------')
    
#     handles, labels = ax1.get_legend_handles_labels()
#     new_handles = [lines.Line2D([], [], c=h.get_edgecolor()) for h in handles]
#     ax1.legend(handles=new_handles, labels=labels)

#     if ate:
#         handles, labels = ax2.get_legend_handles_labels()
#         new_handles = [lines.Line2D([], [], c=h.get_edgecolor()) for h in handles]
#         ax2.legend(handles=new_handles, labels=labels)
#     fig1.suptitle('E1' if 'E1' in folder else 'E2')
#     fig1.show()
    
#     fig_map.suptitle('E1' if 'E1' in folder else 'E2')
    fig_map.show() 
#     fig_exp.suptitle('E1' if 'E1' in folder else 'E2')
    fig_exp.show()        
    
    separator = "-"
    if (saving_results):
        fig_map.savefig('map-{}-{}-{}.eps'.format('E1' if 'E1' in folder else 'E2',separator.join(columns),separator.join(tests)),format='eps',bbox_inches='tight')
        fig_exp.savefig('exp-{}-{}-{}.eps'.format('E1' if 'E1' in folder else 'E2',separator.join(columns),separator.join(tests)),format='eps',bbox_inches='tight')
    
#     fig_area.suptitle('E1' if 'E1' in folder else 'E2')
#     fig_area.show()        
    
    display(df)
    if (saving_results):
        print(df.to_latex(index=True, escape=False))

In [ ]:
#######################################################################################################################
#Author   : Dr. Arun B Ayyar
#
#Based on : Shimazaki H. and Shinomoto S., A method for selecting the bin size of a time histogram Neural Computation (2007)
#	   Vol. 19(6), 1503-1527
#
#Data     : The duration for eruptions of the Old Faithful geyser in Yellowstone National Park (in minutes) 
#	   or normal distribuition.
#	   given at http://176.32.89.45/~hideaki/res/histogram.html
#
#Comments : Implements a faster version than using hist from matplotlib and histogram from numpy libraries	
#           Also implements the shifts for the bin edges
#
########################################################################################################################


def compute_bin_size(data):
    data_max = max(data) #lower end of data
    data_min = min(data) #upper end of data
    n_min = 2500   #Minimum number of bins Ideal value = 2
    n_max = 2501  #Maximum number of bins  Ideal value =200
    n_shift = 1     #number of shifts Ideal value = 30
    N = np.array(range(n_min,n_max))
    D = float(data_max-data_min)/N    #Bin width vector
    Cs = np.zeros((len(D),n_shift)) #Cost function vector
    #Computation of the cost function
    for i in range(np.size(N)):
        shift = np.linspace(0,D[i],n_shift)
        for j in range(n_shift):
            edges = np.linspace(data_min+shift[j]-D[i]/2,data_max+shift[j]-D[i]/2,N[i]+1) # shift the Bin edges
            binindex = np.digitize(data,edges) #Find binindex of each data point
            ki= np.bincount(binindex)[1:N[i]+1] #Find number of points in each bin
            k = np.mean(ki) #Mean of event count
            v = sum((ki-k)**2)/N[i] #Variance of event count
            Cs[i,j]+= (2*k-v)/((D[i])**2) #The cost Function
    C=Cs.mean(1)

    #Optimal Bin Size Selection
    loc = np.argwhere(Cs==Cs.min())[0]
    cmin = C.min()
    idx  = np.where(C==cmin)
    idx = idx[0][0]
    optD = D[idx]
    return np.linspace(data_min+shift[loc[1]]-D[idx]/2,data_max+shift[loc[1]]-D[idx]/2,N[idx]+1,endpoint=False)

In [ ]:
columns = ['S']#,'S_O']#,'S_O','S_DW_O']#['S',]
tests = ['A','INTER_S','OL','NI_NF']#,'OL','I_OL','OL_FIX']#,'A_S','I_NF','NI','NI_NF',
folder = '/home/ebonetto/Desktop/Test'
folder_E2 = os.path.join(folder,'E2')

df_2_ = pd.DataFrame()
a = get_data(folder_E2, columns, tests, df_2_,0,20, True, True)

In [ ]:
columns = ['S']#,'S_O']#,'S_O','S_DW_O']#['S',]
tests = ['OL_I','OL_I_F','OL_FIX','OL_FIX_F']#,'OL','I_OL','OL_FIX']#,'A_S','I_NF','NI','NI_NF',
folder = '/home/ebonetto/Desktop/Test'
folder_E2 = os.path.join(folder,'E2')

df_2_ = pd.DataFrame()
a = get_data(folder_E2, columns, tests, df_2_,0,20, True,True)

In [ ]:
columns = ['S','S_O','S_DW_O']#,'S_O']#,'S_O','S_DW_O']#['S',]
tests = ['A','A_L','OL_I_F']#,'OL','I_OL','OL_FIX']#,'A_S','I_NF','NI','NI_NF',
folder = '/home/ebonetto/Desktop/Test'
folder_E2 = os.path.join(folder,'E2')

df_2_ = pd.DataFrame()
a = get_data(folder_E2, columns, tests, df_2_, 0,20, True, True)